<a href="https://colab.research.google.com/github/mike1336git/colab_notebook/blob/main/with_js/js058_ensembleLJMD3D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#### simulator( html + css + js ) + control( python )

In [1]:
#@title js058_ensembleLJMD3D / def exec_html_js() ... exec me first
#
#  Copyright(C) 2023-2024 Mitsuru Ikeuchi
#  home page: https://mike1336.web.fc2.com/index.html
#  Released under the MIT license ( https://opensource.org/licenses/MIT )
#
#  ver 0.0.0  2023.11.09 created,  last updated on 2024.02.05
#

# def exec_html_js()

import IPython
from IPython.display import display, HTML
from google.colab.output import eval_js

def exec_html_js():
  htm = HTML('''


<!DOCTYPE html>
<html>
<head>
<meta charset="utf-8">
<title>js058_ensembleLJMD3D</title>
<script type="text/javascript">

// %%%%%%%%%%%%%%%%%%%%  javaScript  %%%%%%%%%%%%%%%%%%%%

'use strict';

/* --------------------
//
//  js058_ensembleLJMD3D
//    Copyright(C) 2017-2023 Mitsuru Ikeuchi
//    Released under the MIT license ( https://opensource.org/licenses/MIT )
//
//    ver 0.0.0  2017.12.03 created, last updated on 2018.11.23
//    ver 0.0.1  2019.01.19 v1, last updated on 2021.07.02
//    ver 0.0.2  2021.11.02 v2, last updated on 2021.11.02
//    ver 0.0.3  2023.04.05 v3, last updated on 2023.08.28
//
// --------------------  molecular dynamics 3D
//
//    method: velocity Verlet Algorithm
//      (1) vi = vi + (Fi/mi)*(0.5dt)
//      (2) ri = ri + vi*dt
//      (3) calculation Fi <- {r1,r2,...,rn} Fi=sum(Fij,j=1 to n),Fij=F(ri-rj)
//      (4) vi = vi + (Fi/mi)*(0.5dt)
//      goto (1)
//
//   potential: Lennard-Jones V(r) = 4*epsilon*((sigma/r)^12-(sigma/r)^6)
//           force F(r) = -dV(r)/dr = 24.0*epsilon*r6*(2.0*r6-1.0)/r, r6=(sigma/r)^6
//
//    for faster calculation: O(N) // fast calculation (without pre-registration): O(N^2))
//      ignore F(r) r>rCutoff
//      force F(r) <- force table + linear interpolation (see setForceTable() and cutoff(r))
//      registration with pre-registration
//        pre-registration O(N), see preRegistration()
//          particles pre-regist into lattice section[i][j][k],
//          and register into reg[ni][ip] every particle ni
//        registration reg[][] (see registration()), 'near' means r<rCutoff+20*2000*dt
//          reg[][] use 20 times, assuming particle max speed < 2000m/s
//      force calculation: sum up force(r) (r<rCutoff)
//
//   feature:
//     boundary - wall, periodic
//     ansemble NVE,NVT,NPT
//     Lennard-Jones potential
//
// --------------------
*/

const ensembleLJMD3D = (function(){ // ====================  ensembleLJMD3D Module  ====================

	const g_AMU = 1.66053904e-27;				// (kg) atomic mass unit
	const g_kB = 1.380649e-23;					// (J/K) Boltzmann's constant
	const g_EE = 1.602176634e-19;				// (C) electron charge, energy : 1(eV) = EE(J)
	const g_nMax = 3000;						// array max

	let g_molecKind1 = 2;						// 2:Ar
	let g_molecKind2 = 4;						// 4:Xe
	let g_nKind1 = 500;							// number of kind-1 particles
	let g_nKind2 = 500;							// number of kind-2 particles
	let g_nParticles =1000;						// number of particles
	let g_sysTime = 0.0;						// (s) system time
	let g_timeStep = 5.0*1.0e-15;				// (s) time step dt
	let g_initxMax = 8.0E-9;					// (m) initial x-Box size
	let g_inityMax = 8.0E-9;					// (m) initial y-Box size
	let g_initzMax = 8.0E-9;					// (m) initial z-Box size
	let g_xMax = g_initxMax;					// (m) x-Box size
	let g_yMax = g_inityMax;					// (m) y-Box size
	let g_zMax = g_initzMax;					// (m) z-Box size
	let g_Nsx = 20;								// use pre-registration section(0 to Nsx,0 to Nsx)
	let g_Nsy = 20;								// use pre-registration section(0 to Nsx,0 to Nsy)
	let g_Nsz = 20;								// use pre-registration section(0 to Nsx,0 to Nsy)
	let g_rCutoff = 1.0e-9;						// (m) force cutoff length
	let g_hh = 1.0e-12;							// (m) forceTable r-division
	let g_periodicSW = 1;						// 0:non-periodic 1:periodic

	let g_kineticEnergy = 0.0;					// (J) total kinetic energy
	let g_potentialEnergy = 0.0;				// (J) toal potential energy
	let g_sysTemp = 0.0;						// (K) system temperature
	let g_meanTemp = 300.0;						// (K) mean temperature
	let g_meanPress = 0.0;						// (Pa) mean pressure
	let g_controledTemperatue = 300.0;			// (K) controled temperatue of the system
	let g_controledPressure = 1.0e7;			// (Pa) controled pressure of the system
	let g_xxVirial = 0.0;						// for pressure calc. sum(fxij*xij,{i,j})
	let g_yyVirial = 0.0;						// for pressure calc. sum(fyij*yij,{i,j})
	let g_zzVirial = 0.0;						// for pressure calc. sum(fzij*zij,{i,j})

	const g_xx = dim1( g_nMax );				// (m) x-component of i-th particle position
	const g_yy = dim1( g_nMax );				// (m) y-component of i-th particle position
	const g_zz = dim1( g_nMax );				// (m) z-component of i-th particle position
	const g_vx = dim1( g_nMax );				// (m/s) x-component of i-th particle velocity
	const g_vy = dim1( g_nMax );				// (m/s) y-component of i-th particle velocity
	const g_vz = dim1( g_nMax );				// (m/s) z-component of i-th particle velocity
	const g_ffx = dim1( g_nMax );				// (N) x-component of total force applied i-th particle
	const g_ffy = dim1( g_nMax );				// (N) y-component of total force applied i-th particle
	const g_ffz = dim1( g_nMax );				// (N) z-component of total force applied i-th particle
	const g_mas = dim1( g_nMax );				// (kg) mass of i-th particle
	const g_kind = dimInt1( g_nMax );			// kind of i-th particle
	const g_potentialTable = dim3( 13, 13, 1002 ); // potential table [V[0], V[hh], V[2hh],..., V[rCutoff] ]
	const g_forceTable = dim3( 13, 13, 1002 );	// force table [F[0], F[hh], F[2hh],..., F[rCutoff] ]
	const g_reg = dimInt2( g_nMax, 420 );		// reg[i][0]:total number of particles near i-th particle
												// reg[i][j]:particle number near i-th particle, (j>0)
	const g_section = dimInt4( 40, 40, 40, 20 );// section[i][j][k][0]: total number of particles in the section
												// section[i][j][k][ip]: ip-th particle number in the section

	function dim1( n ) {
		return new Float64Array( n );
	}

	function dimInt1( n ) {
		return new Int32Array( n );
	}

	function dimInt2( ni, nj ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = new Int32Array( nj );
		}
		return a;
	}

	function dim3( ni, nj, nk ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = new Float64Array( nk );
			}
		}
		return a;
	}

	function dimInt4( ni, nj, nk, nl ) {
		let a = [];
		for (let i=0; i<ni; i++) {
			a[i] = [];
			for (let j=0; j<nj; j++) {
				a[i][j] = [];
				for (let k=0; k<nk; k++) {
					a[i][j][k] = new Int32Array( nl );
				}
			}
		}
		return a;
	}


	// --------------------  Lennard-Jones potential data  --------------------

	const g_LJMolec = [
	//    mass(kg)        E(J)         sigma(m)   string  color
		[   4.003*g_AMU,  10.2*g_kB, 2.576e-10, "He",  "#ee4444" ], // 0 He
		[  20.183*g_AMU,  36.2*g_kB, 2.976e-10, "Ne",  "#eeaa00" ], // 1 Ne
		[  39.948*g_AMU, 124.0*g_kB, 3.418e-10, "Ar",  "#aadd00" ], // 2 Ar
		[  83.500*g_AMU, 190.0*g_kB, 3.610e-10, "Kr",  "#44dd44" ], // 3 Kr
		[ 131.300*g_AMU, 229.0*g_kB, 4.055e-10, "Xe",  "#0088ee" ], // 4 Xe
		[ 200.590*g_AMU, 851.0*g_kB, 2.898e-10, "Hg",  "#8800ee" ], // 5 Hg
		[   2.016*g_AMU,  33.3*g_kB, 2.968e-10, "H2",  "#ff8800" ], // 6 H2
		[  28.013*g_AMU,  91.5*g_kB, 3.681e-10, "N2",  "#ff0000" ], // 7 N2
		[  31.999*g_AMU, 113.0*g_kB, 3.433e-10, "O2",  "#ff0088" ], // 8 O2
		[  18.015*g_AMU, 809.1*g_kB, 2.641e-10, "H2O", "#ff00ff" ], // 9 H2O
		[  16.043*g_AMU, 137.0*g_kB, 3.822e-10, "CH2", "#bb00ff" ], //10 CH4
		[  44.010*g_AMU, 190.0*g_kB, 3.996e-10, "CO2", "#6600ff" ], //11 CO2
		[  28.011*g_AMU, 110.0*g_kB, 3.590e-10, "CO",  "#0000ff" ]  //12 CO
	];

	const g_rCollision = [];	// (m) = sigma, V(sigma)==0.0
	const g_rBond = [];			// (m) bond length == 2.0*g_rBond[kind], == sigma*2^(1/6)
	const g_massOf = [];		// (kg) g_massOf[kind] : mass of kind
	const g_strOf = [];			// g_strOf[kind] : string of kind, such as "Ar"
	const g_colorOf = [];		// g_colorOf[kind] : color of kind
	const g_colorStrOf = [];

	(function() {
		const n=g_LJMolec.length;
		for (let i=0; i<n; i++) {
			g_rCollision[i] = g_LJMolec[i][2]*0.5;
			g_rBond[i] = g_LJMolec[i][2]*0.5*1.12246;
			g_massOf[i] = g_LJMolec[i][0];
			g_strOf[i] = g_LJMolec[i][3];
			g_colorOf[i] = g_LJMolec[i][4];
			g_colorStrOf[i] = "<span style='color:"+g_colorOf[i]+"'>"+g_strOf[i]+"</span>"
		}
	}());

	// --- set forceTable[ki][kj][ir] and potentialTable[ki][kj][ir]

	function setForceTable() {
		const hh=g_hh;
		for (let i=0; i<6; i++) {
			for (let j=0; j<6; j++) {
				const ep = Math.sqrt(g_LJMolec[i][1]*g_LJMolec[j][1]);
				const sg = 0.5*(g_LJMolec[i][2]+g_LJMolec[j][2]);
				for (let ir=1; ir<=1001; ir++) {
					const r = ir*hh;
					const ri = (sg/r);
					const r6 = ri*ri*ri*ri*ri*ri;
					g_potentialTable[i][j][ir] = cutoff(r)*4.0*ep*r6*(r6-1.0); // V(r)=4*eps*((sgm/r)^12-(sgm/r)^6)
				}
				g_potentialTable[i][j][0] = g_potentialTable[i][j][1] + g_potentialTable[i][j][2];
				for (let ir=1; ir<=1000; ir++) {
					g_forceTable[i][j][ir] = -(g_potentialTable[i][j][ir+1] - g_potentialTable[i][j][ir-1])/(2.0*hh);
				}
				g_forceTable[i][j][1001] = -(0.0 - g_potentialTable[i][j][1000])/(2.0*hh);
				g_forceTable[i][j][0] = g_forceTable[i][j][1];
			}
		}

		function cutoff(r) {
			let ret;
			if (r>0 && r<0.8*g_rCutoff) {
				ret = 1.0;
			} else if (r>=0.8*g_rCutoff && r<g_rCutoff ) {
				ret = 0.5+0.5*Math.cos(Math.PI*(r-0.8*g_rCutoff)/(0.2*g_rCutoff));
			} else {
				ret = 0.0;
			}
			return ret;
		}
	}


	// --------------------  set initial condition  --------------------

	function setInitialCondition( theme, boundary, contTemp, contPress ) {
		g_periodicSW = boundary;
		g_controledTemperatue = contTemp;
		g_controledPressure = contPress;

		let pSetSW, knd1, knd2, nn1=0, nn2=0, lattice, nLattice=8; //pSetSW 0:crystal 1:liquid 2:gas
		if (theme==0) { //0: Ar(fcc)  knd: 0:He,1:Ne,2:Ar,3:Kr,4:Xe,5:Hg
			pSetSW=0; knd1=2; knd2=2; lattice=5.31e-10*1.0; nLattice=6; // exp.lattice = 5.31
		} else if (theme==1) { //1: Kr(fcc)
			pSetSW=0; knd1=3; knd2=3; lattice=5.64e-10*1.0; nLattice=6; // exp.lattice = 5.64
		} else if (theme==2) { //2: Xe(fcc)
			pSetSW=0; knd1=4; knd2=4; lattice=6.13e-10*1.02; nLattice=6; // exp.lattice = 6.13
		} else if (theme==3) { //3: Ar(liq) -- larger lattice cause to melt the crystal
			pSetSW=1; knd1=2; knd2=2; lattice=5.31e-10*1.2; nLattice=6; // exp.lattice = 5.31
		} else if (theme==4) { //4: Kr(liq)
			pSetSW=1; knd1=3; knd2=3; lattice=5.64e-10*1.2; nLattice=6; // exp.lattice = 5.64
		} else if (theme==5) { //5: Xe(liq)
			pSetSW=1; knd1=4; knd2=4; lattice=6.13e-10*1.2; nLattice=6; // exp.lattice = 6.13
		} else if (theme==6) { //6: Ne
			pSetSW=2; knd1=1; knd2=1; nn1=1000; nn2=0;
		} else if (theme==7) { //7: Ar
			pSetSW=2; knd1=2; knd2=2; nn1=1000; nn2=0;
		} else if (theme==8) { //8: Kr
			pSetSW=2; knd1=3; knd2=3; nn1=1000; nn2=0;
		} else if (theme==9) { //9: Xe
			pSetSW=2; knd1=4; knd2=4; nn1=1000; nn2=0;
		} else if (theme==10) { //10: Hg
			pSetSW=2; knd1=5; knd2=5; nn1=1000; nn2=0;
		} else if (theme==11) { //11: Ar-Xe
			pSetSW=2; knd1=2; knd2=4; nn1=500; nn2=500;
		} else if (theme==12) { //12: Ar-Hg
			pSetSW=2; knd1=2; knd2=5; nn1=500; nn2=500;
		} else if (theme==13) { //13: Ne-Kr
			pSetSW=2; knd1=1; knd2=3; nn1=500; nn2=500;
		} else if (theme==14) { //14: Ne-Ar
			pSetSW=2; knd1=1; knd2=2; nn1=500; nn2=500;
		} else { //else: Ar
			pSetSW=1; knd1=2; knd2=2; nn1=1000; nn2=0;
		}

		g_molecKind1 = knd1;
		g_molecKind2 = knd2;
		g_rCutoff = Math.min(Math.max(6.0*g_rCollision[g_molecKind1],6.0*g_rCollision[g_molecKind2]),1.0e-9);
		let s = 1.0e-9;
		let boxSize, offset, adjustTemp = contTemp;;
		if (pSetSW==0) { // cryatal
			if (g_periodicSW==1) s = 0.0;
			boxSize = lattice*nLattice+s;
			offset = 0.5*s+0.25*lattice; //0.25*lattice;
			adjustTemp = 20.0;
		} else if (pSetSW==1) { // liquid
			boxSize = lattice*nLattice+s;
			offset = 0.5*s+0.25*lattice;
			adjustTemp = 20.0;
		} else if (pSetSW==2) { // gas
			boxSize = 8.0e-9;
		}
		g_initxMax = boxSize;
		g_inityMax = boxSize;
		g_initzMax = boxSize;
		g_xMax = g_initxMax;
		g_yMax = g_inityMax;
		g_zMax = g_initzMax;

		if (pSetSW==0 || pSetSW==1) { // cryatal or liquid
			g_nKind1 = setFCCCrystal(0,knd1,lattice,nLattice,nLattice,nLattice,offset,offset,offset);
			g_nKind2 = 0;
			g_nParticles = g_nKind1+g_nKind2;
		} else if (pSetSW==2) { // gas
			g_nKind1 = nn1;
			g_nKind2 = nn2;
			g_nParticles = setGas(knd1,nn1,knd2,nn2);
		}

		g_sysTime = 0.0;
		setForceTable();
		removeTranslationalMotion();
		ajustVelocity(adjustTemp);
	}

	function setGas(knd1,nn1,knd2,nn2) {
		let nn = nn1 + nn2;
		for (let j=0; j<nn1+nn2; j++) {
			let loopCount = 0, i = 0;
			g_kind[j] = (j<nn1) ? knd1 : knd2;
			do {
				const sgm = 2.0*g_rCollision[g_kind[j]];
				g_xx[j] = (g_xMax-1.2*sgm)*Math.random() + 0.6*sgm;
				g_yy[j] = (g_yMax-1.2*sgm)*Math.random() + 0.6*sgm;
				g_zz[j] = (g_zMax-1.2*sgm)*Math.random() + 0.6*sgm;
				for (i=0; i<j; i++) {
					const d = Math.sqrt((g_xx[i]-g_xx[j])*(g_xx[i]-g_xx[j]) + (g_yy[i]-g_yy[j])*(g_yy[i]-g_yy[j])
										+ (g_zz[i]-g_zz[j])*(g_zz[i]-g_zz[j]));
					if (d < 1.2*(g_rCollision[g_kind[i]]+g_rCollision[g_kind[j]])) break;
				}
				loopCount = loopCount + 1;
				if (loopCount>1000) break;
			} while (i<j);
			if (loopCount>1000) {
				nn = j - 1;
				break;
			}
		}

		for (let i=0; i<nn; i++) {
			setParticle(i,g_kind[i],g_xx[i],g_yy[i],g_zz[i]);
		}
		return nn;
	}

	function setFCCCrystal(ipp,knd,lattice,nx,ny,nz,xPos,yPos,zPos) {
		for (let i=0; i<nx; i++) {
			for (let j=0; j<ny; j++) {
				for (let k=0; k<nz; k++) {
					ipp = setUnitCell(ipp,knd,lattice,i*lattice+xPos,j*lattice+yPos,k*lattice+zPos);
				}
			}
		}
		return ipp;
	}

	function setUnitCell(ipp,knd,lattice,x0,y0,z0) {
		const a = 0.5*lattice;
		setParticle(ipp,knd,x0,y0,z0);
		setParticle(ipp+1,knd,x0+a,y0+a,z0);
		setParticle(ipp+2,knd,x0+a,y0,z0+a);
		setParticle(ipp+3,knd,x0,y0+a,z0+a);
		return ipp+4;
	}

	function setParticle(i,knd,x,y,z) {

		g_xx[i] = x;
		g_yy[i] = y;
		g_zz[i] = z;
		g_kind[i] = knd;
		g_mas[i] =g_massOf[knd];
		g_vx[i] = 200.0*normalRandom3();
		g_vy[i] = 200.0*normalRandom3();
		g_vz[i] = 200.0*normalRandom3();
		g_ffx[i] = 0.0;
		g_ffy[i] = 0.0;
		g_ffz[i] = 0.0;
	}

	// normal distributed random number: -3.0 <= normalRandom3() < 3.0
	function normalRandom3() {
		return (Math.random()+Math.random()+Math.random()+Math.random()+Math.random()+Math.random()-3.0);
	}


	// --------------------  time evolution  --------------------

	function timeEvolution( ensemble, contTemp, contPress ) {
		const nn=g_nParticles;
		if (ensemble==1 || ensemble==2) { // ensemble  0:NVE  1:NVT  2:NPT
			g_controledTemperatue = contTemp;
			if (ensemble==2) g_controledPressure = contPress;
			removeTranslationalMotion();
			ajustVelocity(contTemp);
		}
		if (g_periodicSW==1) {
			registrationPeriodic();
		} else {
			registration();
		}

		for (let ii=0; ii<20; ii++) {
			g_sysTime += g_timeStep;
			if (g_periodicSW==1) {
				moveParticlesPeriodic(g_timeStep);
			} else {
				moveParticles(g_timeStep);
			}
			g_kineticEnergy = 0.0;
			for (let i=0; i<nn; i++) {
				g_kineticEnergy += 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
			}
			g_sysTemp = 2.0*g_kineticEnergy/(3.0*nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
			const press = (nn*g_kB*g_sysTemp+(g_xxVirial+g_yyVirial+g_zzVirial)/3.0)/(g_xMax*g_yMax*g_zMax);
			g_meanTemp = 0.99*g_meanTemp + 0.01*g_sysTemp;
			g_meanPress = 0.99*g_meanPress + 0.01*press;

			if (ensemble==2) volControl(g_controledPressure); // for constant pressure control
		}
	}

	// --- non-periodic

	function moveParticles(dt) {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
			g_zz[i] += g_vz[i]*dt;
		}
		calcForce();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
		}
	}

	function calcForce() {
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0; g_ffz[i]=0;
		}
		let xxvir = 0.0, yyvir = 0.0, zzvir = 0.0;

		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				const xij=g_xx[i]-g_xx[j], yij=g_yy[i]-g_yy[j], zij=g_zz[i]-g_zz[j];
				const rij = Math.sqrt(xij*xij+yij*yij+zij*zij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij, fyij = f*yij/rij, fzij = f*zij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffz[i] += fzij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
					g_ffz[j] -= fzij;
					xxvir +=  fxij*xij;
					yyvir +=  fyij*yij;
					zzvir +=  fzij*zij;
				}
			}
		}
		for (let i=0; i<nn; i++) {
			g_ffx[i] += boundaryForce(g_xx[i]+s05)+boundaryForce(g_xx[i]-g_xMax-s05);
			g_ffy[i] += boundaryForce(g_yy[i]+s05)+boundaryForce(g_yy[i]-g_yMax-s05);
			g_ffz[i] += boundaryForce(g_zz[i]+s05)+boundaryForce(g_zz[i]-g_zMax-s05);
		}
		g_xxVirial = 0.995*g_xxVirial + 0.005*xxvir;
		g_yyVirial = 0.995*g_yyVirial + 0.005*yyvir;
		g_zzVirial = 0.995*g_zzVirial + 0.005*zzvir;
	}

	function boundaryForce(r) {
		const ri = (3.418e-10/r);
		const r6 = ri*ri*ri*ri*ri*ri;
		g_potentialEnergy += 4.0*0.5*1.711e-21*r6*(r6-1.0); // epsilonOfAr = 1.711e-21
		return (24.0*0.5*1.711e-21*r6*(2.0*r6-1.0)/r);
	}

	// registration with preRegistration

	function registration() {
		const nn=g_nParticles;
		preRegistration();
		const rreg = g_rCutoff+20.0*2000.0*g_timeStep;
		const rreg2 = rreg*rreg;
		for (let ipp=0; ipp<nn-1; ipp++) {
			let kp = 1;
			let i0 = Math.floor(g_Nsx*(g_xx[ipp]-rreg)/g_xMax);
			if (i0<0) i0 = 0;
			let i1 = Math.floor(g_Nsx*(g_xx[ipp]+rreg)/g_xMax );
			if (i1>=g_Nsx) i1 = g_Nsx-1;
			let j0 = Math.floor(g_Nsy*(g_yy[ipp]-rreg)/g_yMax );
			if (j0<0) j0 = 0;
			let j1 = Math.floor(g_Nsy*(g_yy[ipp]+rreg)/g_yMax );
			if (j1>=g_Nsy) j1 = g_Nsy-1;
			let k0 = Math.floor(g_Nsz*(g_zz[ipp]-rreg)/g_zMax );
			if (k0<0) k0 = 0;
			let k1 = Math.floor(g_Nsz*(g_zz[ipp]+rreg)/g_zMax );
			if (k1>=g_Nsz) k1 = g_Nsz-1;

			for (let i=i0; i<=i1; i++) {
				for (let j=j0; j<=j1; j++) {
					for (let k=k0; k<=k1; k++) {
						for (let iq=1; iq<=g_section[i][j][k][0]; iq++) {
							const jp = g_section[i][j][k][iq];
							if (jp>ipp) {
								const r2 = (g_xx[ipp]-g_xx[jp])*(g_xx[ipp]-g_xx[jp])
										+ (g_yy[ipp]-g_yy[jp])*(g_yy[ipp]-g_yy[jp])
										+ (g_zz[ipp]-g_zz[jp])*(g_zz[ipp]-g_zz[jp]);
								if (r2<rreg2) {
									g_reg[ipp][kp] = jp;
									kp = kp + 1;
								}
							}
						}
					}
				}
			}
			g_reg[ipp][0] = kp;
		}
		g_reg[nn-1][0] = 0;
	}

	// --- periodic

	function moveParticlesPeriodic(dt) { // periodic
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
			g_xx[i] += g_vx[i]*dt;
			g_yy[i] += g_vy[i]*dt;
			g_zz[i] += g_vz[i]*dt;
		}
		for (let i=0; i<nn; i++) { // periodic condition
			if (g_xx[i]<0.0) g_xx[i] += g_xMax;
			if (g_xx[i]>g_xMax) g_xx[i] -= g_xMax;
			if (g_yy[i]<0.0) g_yy[i] += g_yMax;
			if (g_yy[i]>g_yMax) g_yy[i] -= g_yMax;
			if (g_zz[i]<0.0) g_zz[i] += g_zMax;
			if (g_zz[i]>g_zMax) g_zz[i] -= g_zMax;
		}
		calcForcePeriodic();
		for (let i=0; i<nn; i++) {
			const a = 0.5*dt/g_mas[i];
			g_vx[i] += a*g_ffx[i];
			g_vy[i] += a*g_ffy[i];
			g_vz[i] += a*g_ffz[i];
		}
	}

	function calcForcePeriodic() { // periodic
		const nn=g_nParticles, s05 = 0.5*3.418e-10; // Ar sigma=3.418e-10
		g_potentialEnergy = 0.0;
		for (let i=0; i<nn; i++) {
			g_ffx[i]=0; g_ffy[i]=0; g_ffz[i]=0;
		}
		let xxvir = 0.0, yyvir = 0.0, zzvir = 0.0;

		for (let i=0; i<nn-1; i++) {
			for (let k=1; k<g_reg[i][0]; k++) {
				const j = g_reg[i][k];
				let xij=g_xx[i]-g_xx[j];
				if (xij>0.5*g_xMax) xij -= g_xMax;  //x-periodic
				if (xij<-0.5*g_xMax) xij += g_xMax;
				let yij=g_yy[i]-g_yy[j];
				if (yij>0.5*g_yMax) yij -= g_yMax;  //y-periodic
				if (yij<-0.5*g_yMax) yij += g_yMax;
				let zij=g_zz[i]-g_zz[j];
				if (zij>0.5*g_zMax) zij -= g_zMax;  //z-periodic
				if (zij<-0.5*g_zMax) zij += g_zMax;
				const rij = Math.sqrt(xij*xij+yij*yij+zij*zij);
				if (rij<g_rCutoff) {
					const f = force(rij,g_kind[i],g_kind[j]);
					const fxij = f*xij/rij, fyij = f*yij/rij, fzij = f*zij/rij;
					g_ffx[i] += fxij;
					g_ffy[i] += fyij;
					g_ffz[i] += fzij;
					g_ffx[j] -= fxij;
					g_ffy[j] -= fyij;
					g_ffz[j] -= fzij;
					xxvir +=  fxij*xij;
					yyvir +=  fyij*yij;
					zzvir +=  fzij*zij;
				}
			}
		}
		g_xxVirial = 0.995*g_xxVirial + 0.005*xxvir;
		g_yyVirial = 0.995*g_yyVirial + 0.005*yyvir;
		g_zzVirial = 0.995*g_zzVirial + 0.005*zzvir;
	}

	// registration with preRegistration

	function registrationPeriodic() { // periodic
		const nn=g_nParticles;
		preRegistration();

		const rreg = g_rCutoff + 2000.0*20.0*g_timeStep;
		const rreg2 = rreg*rreg;
		for (let ip=0; ip<nn-1; ip++) {
			let kp = 1;
			const i0 = Math.floor(g_Nsx*(g_xx[ip]-rreg)/g_xMax+g_Nsx)-g_Nsx;
			const i1 = Math.floor(g_Nsx*(g_xx[ip]+rreg)/g_xMax);
			const j0 = Math.floor(g_Nsy*(g_yy[ip]-rreg)/g_yMax+g_Nsy)-g_Nsy;
			const j1 = Math.floor(g_Nsy*(g_yy[ip]+rreg)/g_yMax);
			const k0 = Math.floor(g_Nsz*(g_zz[ip]-rreg)/g_zMax+g_Nsz)-g_Nsz;
			const k1 = Math.floor(g_Nsz*(g_zz[ip]+rreg)/g_zMax);
			for (let i=i0; i<=i1; i++) {
				const ii = (i+g_Nsx)%g_Nsx;
				for (let j=j0; j<=j1; j++) {
					const jj = (j+g_Nsy)%g_Nsy;
					for (let k=k0; k<=k1; k++) {
						const kk = (k+g_Nsz)%g_Nsz;
						for (let iq=1; iq<=g_section[ii][jj][kk][0]; iq++) {
							const jp = g_section[ii][jj][kk][iq];
							if (jp>ip) {
								let xij=g_xx[ip]-g_xx[jp];
								if (xij>0.5*g_xMax) xij -= g_xMax;  //x-periodic
								if (xij<-0.5*g_xMax) xij += g_xMax;
								let yij=g_yy[ip]-g_yy[jp];
								if (yij>0.5*g_yMax) yij -= g_yMax;  //y-periodic
								if (yij<-0.5*g_yMax) yij += g_yMax;
								let zij=g_zz[ip]-g_zz[jp];
								if (zij>0.5*g_zMax) zij -= g_zMax;  //z-periodic
								if (zij<-0.5*g_zMax) zij += g_zMax;
								const r2 = xij*xij+yij*yij+zij*zij;
								if (r2<rreg2) {
									g_reg[ip][kp]=jp;
									kp += 1;
								}
							}
						}
					}
				}
			}
			g_reg[ip][0]=kp;
		}
		g_reg[nn-1][0] = 0;
	}

	// --- common functions non-periodic, periodic

	function force(r,ki,kj) { // forceTable - linear interporation
		const hh=g_hh;
		const ir = Math.floor(r/hh);
		const a = r - ir*hh;
		g_potentialEnergy += ((hh-a)*g_potentialTable[ki][kj][ir] + a*g_potentialTable[ki][kj][ir+1])/hh;
		return ((hh-a)*g_forceTable[ki][kj][ir] + a*g_forceTable[ki][kj][ir+1])/hh;
	}

	function preRegistration() {
		const nn=g_nParticles, nsx=g_Nsx, nsy=g_Nsy, nsz=g_Nsz;
		for (let i=0; i<nsx; i++) {
			for (let j=0; j<nsy; j++) {
				for (let k=0; k<nsz; k++) {
					g_section[i][j][k][0] = 0;
				}
			}
		}
		for (let ipp=0; ipp<nn; ipp++) {
			let i = Math.floor(nsx*g_xx[ipp]/g_xMax);
			if (i>=nsx) i = nsx-1;
			if (i<0) i = 0;
			let j = Math.floor(nsy*g_yy[ipp]/g_yMax);
			if (j>=nsy) j = nsy-1;
			if (j<0) j = 0;
			let k = Math.floor(nsz*g_zz[ipp]/g_zMax);
			if (k>=nsz) k = nsz-1;
			if (k<0) k = 0;
			const iq = g_section[i][j][k][0] + 1;
			g_section[i][j][k][0] = iq;
			g_section[i][j][k][iq] = ipp
		}
	}

	function maxNearParticles() {
		const nn=g_nParticles;
		let mx=0;
		for (let i=0; i<nn-1; i++) {
			if (mx<g_reg[i][0]) mx = g_reg[i][0];
		}
		return (mx-1);
	}

	// --- volume control

	function volControl(contPress) {
		const d = g_timeStep*30.0; // max wall speed = 30 m/s
		const a = 1.0e-12;

		let x = (1.0+a*(xxPress()-contPress))*g_xMax;
		if (x>g_xMax+d) {
			x = g_xMax + d;
		} else if (x<g_xMax-d) {
			x = g_xMax - d;
		}
		const xShift = 0.5*(x - g_xMax);
		g_xMax = x;

		let y = (1.0+a*(yyPress()-contPress))*g_yMax;
		if (y>g_yMax+d) {
			y = g_yMax + d;
		} else if (y<g_yMax-d) {
			y = g_yMax - d;
		}
		const yShift = 0.5*(y - g_yMax);
		g_yMax = y;

		let z = (1.0+a*(zzPress()-contPress))*g_zMax;
		if (z>g_zMax+d) {
			z = g_zMax + d;
		} else if (z<g_zMax-d) {
			z = g_zMax - d;
		}
		const zShift = 0.5*(z - g_zMax);
		g_zMax = z;

		shiftParticles( xShift, yShift, zShift );
	}

	function shiftParticles( xShift, yShift, zShift ) {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g_xx[i] += xShift;
			g_yy[i] += yShift;
			g_zz[i] += zShift;
		}
	}


	// --- pressure

	function pressure() {
		const vol=g_xMax*g_yMax*g_zMax;
		return (g_nParticles*g_kB*g_meanTemp+(g_xxVirial+g_yyVirial+g_zzVirial)/3.0)/vol;
	}

	function xxPress() { return dirPress(g_vx, g_vx, g_xxVirial); };

	function yyPress() { return dirPress(g_vy, g_vy, g_yyVirial); };

	function zzPress() { return dirPress(g_vz, g_vz, g_zzVirial); };

	function dirPress(v1, v2, virial) {
		const nn=g_nParticles;
		let nkt= 0.0;
		for (let i=0; i<nn; i++) {
			nkt += g_mas[i]*v1[i]*v2[i];
		}
		return (nkt+virial)/(g_xMax*g_yMax*g_zMax);
	}

	// --- utility

	function systemTemperature() {
		const nn=g_nParticles;
		let ek=0.0;  //kinetic energy (J)
		for (let i=0; i<nn; i++) {
			ek = ek + 0.5*g_mas[i]*(g_vx[i]*g_vx[i]+g_vy[i]*g_vy[i]+g_vz[i]*g_vz[i]);
		}
		return 2.0*ek/(3.0*nn*g_kB); //2D: E/N=kT, 3D: E/N=(3/2)kT
	}

	function pressure() {
		const vol = g_xMax*g_yMax*g_zMax;
		return (g_nParticles*g_kB*systemTemperature()+(g_xxVirial+g_yyVirial+g_zzVirial)/3.0)/vol;
	}

	function ajustVelocity(temp) {
		const nn=g_nParticles;
		const a = Math.sqrt(temp/systemTemperature());
		for (let i=0; i<nn; i++) {
			g_vx[i] = a*g_vx[i];
			g_vy[i] = a*g_vy[i];
			g_vz[i] = a*g_vz[i];
		}
	}

	function removeTranslationalMotion() {
		const nn=g_nParticles;
		let m=0.0, mvx=0.0, mvy=0.0, mvz=0.0;
		for (let i=0; i<nn; i++) {
			m += g_mas[i];
			mvx += g_mas[i]*g_vx[i];
			mvy += g_mas[i]*g_vy[i];
			mvz += g_mas[i]*g_vz[i];
		}
		const vtx = mvx/m, vty = mvy/m, vtz = mvz/m;
		for (let i=0; i<nn; i++) {
			g_vx[i] -= vtx;
			g_vy[i] -= vty;
			g_vz[i] -= vtz;
		}
	}


	// --------------------  public  --------------------

	return {
		init:			setInitialCondition,	// setInitialCondition( theme, boundary, contTemp, contPress )
		evolve:			timeEvolution,			// timeEvolution( ensemble, contTemp, contPress )

		setTemp:		ajustVelocity,			// ajustVelocity( temp )
		//setBonds:		setBonds,				// setBonds()

		getSysParam:	function() { return [ g_molecKind1, g_molecKind2, g_nParticles, g_timeStep,
												g_initxMax, g_inityMax, g_initzMax ]; },
		getnKinds:		function() { return [ g_nKind1, g_nKind2 ]; },
		getBoxSize:		function() { return [ g_xMax, g_yMax, g_zMax ]; }, // change boxsize in NPT condition
		getNow:			function() { return [ g_sysTime, g_meanTemp, g_kineticEnergy, g_potentialEnergy, g_meanPress ]; },

		getBondLength0:	function(ki,kj) { return g_rBond[ki]+g_rBond[kj]; },
		getPotentialTable:	function(ki,kj) { return g_potentialTable[ki][kj]; },
		getForceTable:	function(ki,kj) { return g_forceTable[ki][kj]; },
		getKindStr:		function(kind) { return g_strOf[kind]; },
		getKindmass:	function(kind) { return g_massOf[kind]; },
		getKindrCollision:	function(kind) { return g_rCollision[kind]; },

		getKind:		function(i) { return g_kind[i] },
		getMolecData:	function(i) { const kind = g_kind[i]; return [ g_rCollision[kind], g_colorOf[kind] ]; },
		getNearList:	function(i) { return g_reg[i]; },
		getPosition:	function(i) { return [ g_xx[i], g_yy[i], g_zz[i] ]; },
		getVelocity:	function(i) { return [ g_vx[i], g_vy[i], g_vz[i] ]; },
		getForce:		function(i) { return [ g_ffx[i], g_ffy[i], g_ffz[i] ]; },
		//getBonds:		function(iBond) { return g_bondArray[iBond]; },
	};

})(); // ====================  ensembleLJMD3D end  ====================


const js058 = (function(){ // ====================  js Module  ====================

	const theModule = ensembleLJMD3D;
	// theme number           0    1    2    3    4    5    6    7    8    9   10   11   12   13   14
	const contTempData  = [  20,  20,  20,  50,  50,  50, 300, 300, 300, 300, 300, 300, 300, 300, 300, ]; // (K)
	const contPressData = [  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10,  10, ]; // (MPa)
	const themeStr = [ "0:Ar(fcc)","1:Kr(fcc)","2:Xe(fcc)", "3:Ar(liq)","4:Kr(fliq)","5:Xe(fliq)",
						"6:Ne","7;Ar","8:Kr","9:Xe","10:Hg", "11:Ar-Xe","12:Ar-Hg","13:Ne-Kr","14:Ne-Ar" ];
	// kind number: 0:He, 1:Ne, 2:Ar, 3:Kr, 4:Xe, 5;Hg
	const ballColorData = [ /*0:He*/ [ 255,  80,  80 ], /*1:Ne*/ [ 255, 180,   0 ], /*2:Ar*/ [ 180, 255,   0 ],
							/*3:Kr*/ [  80, 255,  80 ], /*4:Xe*/ [  80, 180, 255 ], /*5:Hg*/ [ 128,   0, 255 ] ];

	const xCanvasSize = 480;	// in pixel
	const yCanvasSize = 480;	// in pixel
	let canvas;
	let ctx;

	let v_theme = 7;			// 7: Ar gas
	let v_boundary = 0;			// (==g_periodicSW)  0:non-periodic 1:periodic
	let v_contTemp = 300.0;		// (K) controled temperature
	let v_contPress = 1.0e7;	// (Pa) controled pressure
	let v_ensemble = 0;			// ensemble 0:NVE 1:NVT 2:NPT

	let p_kind1, p_kind2, p_nParticles, p_dt, p_initxMax, p_inityMax, p_initzMax; // <-- theModule.getSysParam()
	let p_xMax, p_yMax, p_zMax; // <-- theModule.getBoxSize()
	let p_nKind1, p_nKind2; // <-- theModule.getnKinds()
	let sysTime, meanTemp, kineticEnergy, potentialEnergy, meanPress;
	let nowData = [];
  let kindList = [];
  let xxList = [];
	let yyList = [];
  let zzList = [];
	let vxList = [];
	let vyList = [];
  let vzList = [];

	let dispMode = 0;
	let nCalc = 1;
	let resetFlag = true;
	let pauseFlag = false;
	let stepFlag = false;
	let inStepFlag = false;

  let breakFlag = false;
	let perticleFlag = true;

	function main() {
		resetFlag = true;
		setCanvas( 'canvas_box', xCanvasSize, yCanvasSize );
		initDom();

		animate();

		function setCanvas( canvasID, width, height ) {
			canvas = document.getElementById( canvasID );
			canvas.width  = width;
			canvas.height = height;
			ctx = canvas.getContext('2d');
			ctx.font = "16px 'sans-serif'";
			ctx.textBaseline = "bottom";
			ctx.textAlign = "left";
			ctx.lineWidth = 1;
			g3d.setMouseOnCanvas( canvas ); // 3D graphics
		}
	}


	function animate() {
    if ( breakFlag ) return;

		if ( resetFlag ) {
			resetFlag = false;
			v_contTemp = contTempData[ v_theme ];
			v_contPress = contPressData[ v_theme ]*1.0e6;
			theModule.init( v_theme, v_boundary, v_contTemp, v_contPress );
			[ p_kind1, p_kind2, p_nParticles, p_dt, p_initxMax, p_inityMax, p_initzMax ] = theModule.getSysParam();
			[ p_nKind1, p_nKind2 ] = theModule.getnKinds();
			document.getElementById("range_temp").value = v_contTemp;
			document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
      perticleFlag = true;
		}

		if ( !pauseFlag ) {
			for (let i=0; i<nCalc; i++) {
				theModule.evolve( v_ensemble, v_contTemp, v_contPress );
			}
		} else if ( pauseFlag && stepFlag ) {
			stepFlag = false;
			theModule.evolve( v_ensemble, v_contTemp, v_contPress );
			inStepFlag = true;
		}

		draw( ctx, dispMode );

		if ( perticleFlag ) setParticlesData();

		requestAnimationFrame(animate);
	}

  function setParticlesData() {
		nowData = [ sysTime, meanTemp, kineticEnergy, potentialEnergy, meanPress ];
    for (let i=0; i<p_nParticles; i++) {
			let knd, x, y, z, vx, vy, vz;
      knd = theModule.getKind(i);
      kindList[i] = knd;
			[ x, y, z ] = theModule.getPosition(i);
			xxList[i] = x;
			yyList[i] = y;
      zzList[i] = z;
			[ vx, vy, vz ] = theModule.getVelocity(i);
			vxList[i] = vx;
			vyList[i] = vy;
      vzList[i] = vz;
		}
	}


	// --------------------  draw  --------------------

	function draw( ctx, dispMode ) {
		[ p_xMax, p_yMax, p_zMax ] = theModule.getBoxSize();
		[ sysTime, meanTemp, kineticEnergy, potentialEnergy, meanPress ] = theModule.getNow();
		const str1 = theModule.getKindStr(p_kind1), str2 = theModule.getKindStr(p_kind2);
		//const xBoxSize = 300, xPos = 90, yPos = 55, scale = xBoxSize/p_initxMax; // g3d param
		const xp = 40, yp = 5, yTextPos= 430;

		// clear
		ctx.clearRect(0, 0, xCanvasSize, yCanvasSize);

		// draw
		// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
		g3d.init( p_nParticles, p_xMax, p_yMax, p_zMax, xCanvasSize, yCanvasSize, 300, 20 );
		if ( dispMode==0 ) { // ball
			//draw3D( ctx, nParticles, ballScale, mode ) // mode 0:ball, 1:bond, 2:velocity
			draw3D( ctx, p_nParticles, 1.0, 0 );
		} else if ( dispMode==1 ) { // small ball
			draw3D( ctx, p_nParticles, 0.4, 0 );
		} else if ( dispMode==2 ) { // small ball + bond
			draw3D( ctx, p_nParticles, 0.4, 1 );
		} else if ( dispMode==3 ) { // bond only
			draw3D( ctx, p_nParticles, 0.0, 1 );
		} else if ( dispMode==4 ) { // ball + velocity
			draw3D( ctx, p_nParticles, 0.3, 2 );

		} else if ( dispMode==5 ) { // velocity space
			const vFullScale = 1500.0; // (m/s)
			drawVelocitySpace(ctx, vFullScale );

		} else if ( dispMode==6 ) { // velocity distribution function
			if ( !pauseFlag || inStepFlag ) pfg.setVDF( p_nParticles, theModule.getVelocity );
			inStepFlag = false;
			const mass1 = theModule.getKindmass(p_kind1), mass2 = theModule.getKindmass(p_kind2);
			pfg.drawVelocityDistribution( ctx, str1, mass1, p_nKind1, str2, mass2, p_nKind2, meanTemp, xp, yp );

		} else if ( dispMode==7 ) { // vradial distribution function
			if ( !pauseFlag || inStepFlag ) pfg.setRDF( p_nParticles, theModule.getPosition, theModule.getNearList );
			inStepFlag = false;
			pfg.drawRadialDistribution(ctx,xp,yp);

		} else if ( dispMode==8 ) { // force F(r)
			const magFactor = 40.0;
			//pfg.drawInteraction( ctx, tableKind, kind1, str1, kind2, str2, tableFunc, magFactor, xp, yp );
			pfg.drawInteraction( ctx, 1, p_kind1, str1, p_kind2, str2, theModule.getForceTable, magFactor, xp, yp );

		} else if ( dispMode==9 ) { // potential V(r)
			const magFactor = 40.0;
			//pfg.drawInteraction( ctx, tableKind, kind1, str1, kind2, str2, tableFunc, magFactor, xp, yp );
			pfg.drawInteraction( ctx, 0, p_kind1, str1, p_kind2, str2, theModule.getPotentialTable, magFactor, xp, yp );
		}

		// caption
		const molecStr = (p_kind1==p_kind2) ? str1 : str1 + "," + str2;
		ctx.fillStyle = "#888888";
		ctx.fillText(`time = ${(sysTime*1.0e12).toFixed(1)} (ps)`, 20, yTextPos);
		ctx.fillText(`Temp = ${meanTemp.toFixed(1)} (K)`, 240, yTextPos);
		ctx.fillText(`molec:${molecStr}, N = ${p_nParticles}`, 20, yTextPos+20);
		ctx.fillText(`Press = ${(meanPress*1.0e-6).toFixed(3)} (MPa)`, 240, yTextPos+20);
		ctx.fillText(`Box = ${(p_xMax*1.0e9).toFixed(3)}x${(p_yMax*1.0e9).toFixed(3)}x${(p_zMax*1.0e9).toFixed(3)} (nm)`,
						20, yTextPos+40);
		ctx.fillText(`Energy = ${(kineticEnergy+potentialEnergy).toExponential(4)} (J)`, 280, yTextPos+40);
		//document.getElementById("text_caption").innerHTML = "minimum molecular dynamics code"
	}

	function draw3D( ctx, nParticles, ballScale, mode ) {
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc(); // g3d sc, xp, yp
		const nn = nParticles, vmag = 200.0*p_dt;

		g3d.set3DRotatedObjects(0.0); //(rotateRateOfAyInDegree) eg. 0.0 or 0.5
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444400"); // dark yellow
		for (let i=0; i<nn; i++) {
			const j = g3d.srtzix[i];
			const zr = (g3d.ppz[j]/g_zMax+1.27)/3.0; // -0.73 < g3d.ppz[j]/g_zMax < 1.73
			const kind = theModule.getKind(j);
			const rCollision = theModule.getKindrCollision(kind);
			const col = ballColor( kind, zr );
			const x = g3d.ppx[j]*sc+xp, y = g3d.ppy[j]*sc+yp, r = rCollision*sc*ballScale;
			if ( r>0.0 ) {
				g3d.drawDisc( ctx, x, y, r, col ); // (x, y, r, color)
				g3d.strokeCircle( ctx, x, y, r, ballColor( kind, 0.7*zr ) );
			}
			if ( mode==0 ) continue;
			if ( mode==1 ) {
				const reg = theModule.getNearList(j);
				const n = reg[0];
				let x1, y1, z1; [ x1, y1, z1 ] = theModule.getPosition(j);
				for (let k=1; k<n; k++) {
					const jj = reg[k];
					const kindjj = theModule.getKind(jj)
					let x2, y2, z2; [ x2, y2, z2 ] = theModule.getPosition(jj);
					const d = Math.sqrt( (x1-x2)*(x1-x2) + (y1-y2)*(y1-y2) + (z1-z2)*(z1-z2) );
					const d0 = theModule.getBondLength0(kind,kindjj);
					if ( d<d0*1.2 ) {
						g3d.drawLine(ctx, x, y, g3d.ppx[jj]*sc+xp, g3d.ppy[jj]*sc+yp, "#444444" );
					}
				}
			} else if ( mode==2 ) {
				let x1, y1, z1, vx, vy, vz, x2, y2;
				[ x1, y1, z1 ] = theModule.getPosition(j);
				[ vx, vy, vz ] = theModule.getVelocity(j);
				[ x2, y2 ] = g3d.rotateXYPos( x1+vx*vmag, y1+vy*vmag, z1+vz*vmag );
				g3d.drawLine(ctx, x,y, x2*sc+xp, y2*sc+yp, "#dd8888");
			}
		}
		g3d.plotNearEdge(ctx, sc,xp,yp,"#999900"); // yellow
		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#888888";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	}

	function ballColor( kind, zr ) {
		const r = Math.floor(ballColorData[kind][0]*zr);
		const g = Math.floor(ballColorData[kind][1]*zr);
		const b = Math.floor(ballColorData[kind][2]*zr);
		return "rgb("+r+","+g+","+b+")";
	}


	function drawVelocitySpace( ctx, vFullScale ) {
		const boxSize = 300;
		g3d.init( p_nParticles, vFullScale*2, vFullScale*2, vFullScale*2, xCanvasSize, yCanvasSize, 300, 20 );
		let sc, xp, yp; [ sc, xp, yp ] = g3d.scxpypFunc(); // g3d sc, xp, yp
		const nn = p_nParticles, r = 20.0*sc;

		g3d.setVspaceRotatedObjects(0.0);
		g3d.plotFarEdge(ctx, sc,xp,yp,"#444444"); // dark gray
		for (let i=0; i<nn; i++) {
			const j = g3d.srtzix[i];
			const zr = (g3d.ppz[j]/g_zMax+1.27)/3.0; // -0.73 < g3d.ppz[j]/g_zMax < 1.73
			const kind = theModule.getKind(j);
			const rCollision = theModule.getKindrCollision(kind);
			const x = g3d.ppx[j]*sc+xp, y = g3d.ppy[j]*sc+yp;
			g3d.strokeCircle( ctx, x, y, r, ballColor( kind, zr ) );
		}
		g3d.plotNearEdge(ctx, sc,xp,yp,"#999999"); // gray
		ctx.font = "12px 'sans-serif'";
		ctx.fillStyle = "#666666";
		ctx.fillText(`Ax=${(g3d.Ax*180/Math.PI).toFixed(1)}, Ay=${(g3d.Ay*180/Math.PI).toFixed(1)}`, 10, 15);
		ctx.font = "16px 'sans-serif'";
	}


	//--------------------  pfg 3D ( plot function graphics) module  --------------------
	// ver 0.0.2  2020.10.14
	// ver 0.0.3  2021.06.24  last updated on 2021.07.02
	// ver 0.0.4  2023.04.06  last updated 2023.04.09

	const pfg = {};					// namespace of plot function graphics module
	pfg.vdiv = 5.0;					// (m/s) velocity division
	pfg.n = 400;					// number of v-division
	pfg.irMax = 1000;//1200;		// =1000(for LJ,Morse) or 1200(for ion-3D)
	pfg.kB = 1.380649e-23;			// (J/K) Boltzmann's constant
	pfg.EE = 1.602176634e-19;		// (C) electron charge, energy : 1(eV) = EE(J)
	pfg.yFigureSize = 400;			// y-figure size

	pfg.vdf = pfg_dim1(pfg.n);		// velocity distribution function, vdf[iv]
	pfg.tempVD = pfg_dim1(pfg.n);	// temporal verocity distribution function, tempVD[iv]
	pfg.Maxwell = pfg_dim1(pfg.n);	// Maxwell distribution, Maxwell[iv]
	pfg.rdf = pfg_dim1(100);		// radial distribution function, rdf[ir]
	pfg.tempRD = pfg_dim1(100);		// temporal radial distribution function, tempRD[ir]

	pfg.colr = {
		black:"rgb(0,0,0)",gray:"rgb(80,80,80)",lightGray:"rgb(120,120,120)",white:"rgb(250,250,250)",
		red:"rgb(250,0,0)",green:"rgb(0,250,0)",blue:"rgb(0,0,250)",
		cyan:"rgb(0,250,250)",yellow:"rgb(250,250,0)",magenta:"rgb(250,0,250)" };

	function pfg_dim1(n) {
		let a=[];
		for (let i=0; i<n; i++) {
			a[i] = 0.0;
		}
		return a;
	}

	// --- vdf 3d

	pfg.setVDF = function( nParticles, velocityFunc ) {
		const vdiv=pfg.vdiv, n=pfg.n;
		for (let iv=0; iv<n; iv++) {
			pfg.tempVD[iv] = 0;
		}
		for (let i=0; i<nParticles; i++) {
			let vx, vy, vz;
			[ vx, vy, vz ] = velocityFunc(i);
			const v = Math.sqrt(vx*vx + vy*vy + vz*vz);
			const iv = Math.floor(v/vdiv+0.5);
			if ( iv<n ) {
				pfg.tempVD[iv] += 1.0/nParticles;
			}
		}
		for (let iv=0; iv<n; iv++) {
			pfg.vdf[iv] = 0.99*pfg.vdf[iv]+0.01*pfg.tempVD[iv];
		}
	};

	pfg.setMaxwell = function( mass, temp ) { // f(v) = (4*pi*v*v)*(m/(2*pi*k*T))^1.5*exp(-m*v*v/(2*k*T))
		const n=pfg.n, vdiv=pfg.vdiv, kB=pfg.kB, pi=Math.PI;
		const kT = kB*temp;
		const a = 4*pi*Math.pow((mass/(2*pi*kT)),1.5);
		const b = mass/(2*kT);
		for (let iv=0; iv<n; iv++) {
			const v = iv*vdiv;
			pfg.Maxwell[iv] = vdiv*v*v*a*Math.exp(-b*v*v);
		}
	};

	pfg.setMixedMaxwell = function(mass1,nn1,mass2,nn2,temp) {
		const vdiv=pfg.vdiv, n=pfg.n, kB=pfg.kB, pi=Math.PI;
		const kT = kB*temp;
		const a1 = 4*pi*Math.pow((mass1/(2*pi*kT)),1.5);
		const b1 = mass1/(2*kT);
		const a2 = 4*pi*Math.pow((mass2/(2*pi*kT)),1.5);
		const b2 = mass2/(2*kT);
		for (let iv=0; iv<n; iv++) {
			const v = iv*vdiv;
			const f1 = vdiv*v*v*a1*Math.exp(-b1*v*v);
			const f2 = vdiv*v*v*a2*Math.exp(-b2*v*v);
			pfg.Maxwell[iv] = (nn1/(nn1+nn2))*f1 + (nn2/(nn1+nn2))*f2;
		}
	};

	pfg.drawVelocityDistribution = function( ctx, str1, mass1, nn1, str2, mass2, nn2, temp, xp, yp ) {
		const ySize=pfg.yFigureSize, mag=10000.0, col=pfg.colr;
		let ypp = yp+20;

		//grid
		pfg.drawVGrid(ctx,xp,yp);

		//plot
		pfg.setMaxwell(mass1,temp);
		pfg.drawVDF(ctx,xp,ySize,pfg.Maxwell,mag,col.yellow);
		if ( str1 !=str2 ) {
			pfg.setMaxwell(mass2,temp);
			pfg.drawVDF(ctx,xp,yp,pfg.Maxwell,mag,col.cyan);
			pfg.setMixedMaxwell(mass1,nn1,mass2,nn2,temp);
			pfg.drawVDF(ctx,xp,yp,pfg.Maxwell,mag,col.magenta);
		}
		pfg.drawVDF(ctx,xp,yp,pfg.tempVD,mag,col.lightGray);
		pfg.drawVDF(ctx,xp,yp,pfg.vdf,mag,col.red);

		//caption
		pfg.drawText(ctx,"velocity distribution function", xp+50, ySize-24, col.gray);
		pfg.drawText(ctx,"Maxwell distr. :"+str1, xp+50, ypp, col.yellow); ypp+=20;
		if ( str1 != str2 ) {
			pfg.drawText(ctx,"Maxwell distr. :"+str2, xp+50, ypp, col.cyan); ypp+=20;
			pfg.drawText(ctx,"mixed Maxwell distr.", xp+50, ypp, col.magenta); ypp+=20;
		}
		pfg.drawText(ctx,"mean velocity distribution", xp+50, ypp, col.red); ypp+=20;
		pfg.drawText(ctx,"velocity distribution at present", xp+50, ypp, col.lightGray);
	};

	pfg.drawVDF = function( ctx, xp, yp, vd, mag, color ) {
		const ySize=pfg.yFigureSize;
		ctx.strokeStyle = color;
		ctx.beginPath();
		for (let iv=0; iv<300; iv++) {
			ctx.lineTo(xp+iv, ySize-60-mag*vd[iv]);
		}
		ctx.stroke();
	};

	pfg.drawVGrid = function(ctx,xp,yp) {
		const ySize=pfg.yFigureSize, col=pfg.colr;
		for (let i=0; i<=300; i+=100) {
			pfg.drawLine(ctx,xp+i,yp, xp+i,ySize-60, col.lightGray);
			pfg.drawText(ctx,i*5, xp+i-15, ySize-60+20,col.gray);
		}
		ctx.strokeStyle = col.gray;
		ctx.strokeRect(xp,yp,300,ySize-yp-60);
		pfg.drawText(ctx,"(m/s)", xp+335, ySize-60+20,col.gray);
	};

	//--- rdf 3D

	pfg.setRDF = function( nParticles, positionFunc, nearListFunc ) {
		const nn=nParticles, dr=1.0e-11, r0=3.0e-10;
		const a = 4*Math.PI*dr*dr*dr/(r0*r0*r0);
		for (let ir=0; ir<100; ir++) {
			pfg.tempRD[ir] = 0.0;
		}
		for (let i=0; i<nn-1; i++) {
			const near = nearListFunc(i);
			for (let k=1; k<near[0]; k++) {
				const j = near[k];
				let xi, yi, zi, xj, yj, zj;
				[ xi, yi, zi ] = positionFunc(i);
				[ xj, yj, zj ] = positionFunc(j);
				const xij=xi-xj, yij=yi-yj, zij=zi-zj;
				const rij = Math.sqrt(xij*xij+yij*yij+zij*zij);
				if (rij<1.0e-9) {
					const ir = Math.floor(rij*1.0e+11+0.5);
					pfg.tempRD[ir] += 1.0/nn/(a*ir*ir);
				}
			}
		}
		for (let ir=0; ir<100; ir++) {
			pfg.rdf[ir] = 0.99*pfg.rdf[ir]+0.01*pfg.tempRD[ir];
		}
	};

	pfg.drawRadialDistribution = function( ctx, xp, yp ) {
		const ySize=pfg.yFigureSize, col=pfg.colr;

		//grid
		pfg.drawRGrid(ctx,xp,yp);

		//plot
		pfg.drawRDF(ctx,xp,yp,pfg.tempRD,col.lightGray);
		pfg.drawRDF(ctx,xp,yp,pfg.rdf,col.red);

		//caption
		pfg.drawText(ctx,"radial distribution function", xp+50, ySize-24,col.gray);
		pfg.drawText(ctx,"mean radial distribution", xp+50, yp+20,col.red);
		pfg.drawText(ctx,"radial distribution at present", xp+50, yp+40,col.lightGray);
	};

	pfg.drawRDF = function(ctx,xp,yp,rd,color) {
		const yHeight=(pfg.yFigureSize-80), mag=100.0;
		ctx.strokeStyle = color;
		ctx.beginPath();
		for (let ir=0; ir<100; ir++) {
			ctx.lineTo(xp+ir*3, yp+yHeight-mag*rd[ir]);
		}
		ctx.stroke();
	};

	pfg.drawRGrid = function(ctx,xp,yp) {
		const yHeight=(pfg.yFigureSize-80), yBottom=yp+yHeight, col=pfg.colr;
		ctx.fillStyle = col.gray;
		for (let i=0; i<=300; i+=60) {
			pfg.drawLine(ctx,xp+i,yp, xp+i, yp+yHeight, col.lightGray);
		}
		ctx.strokeStyle = col.gray;
		ctx.strokeRect(xp,yp,300,yHeight);
		ctx.fillText("0", xp-5, yBottom+20);
		ctx.fillText("1.0(nm)", xp+300-10, yBottom+20);
	};

	//--- V(r), F(r)

	pfg.drawInteraction = function( ctx, tableKind, kind1, str1, kind2, str2, tableFunc, magFactor, xp, yp ) {
		const yHeight=(pfg.yFigureSize-80), yBottom=yp+yHeight, y0=yp+180, col=pfg.colr;
		let mag, str;
		if (tableKind==0) { // potential
			mag = magFactor*30.0/pfg.EE;
			str = "interaction potential V(r)";
		} else if (tableKind==1) { // force
			mag = magFactor*3.0e10;
			str = "interaction force F(r)";
		}

		//grid
		pfg.drawRGrid(ctx,xp,yp);
		pfg.drawLine(ctx,xp, y0, xp+300, y0, col.lightGray);
		pfg.drawText(ctx, "0", xp-12, y0+8, col.gray);

		//plot
		if ( kind1==kind2 ) {
			pfg.drawTable(ctx,xp,yp,y0,tableFunc(kind1,kind1),mag,col.cyan);
		} else {
			pfg.drawTable(ctx,xp,yp,y0,tableFunc(kind1,kind1),mag,col.cyan);
			pfg.drawTable(ctx,xp,yp,y0,tableFunc(kind1,kind2),mag,col.green);
			pfg.drawTable(ctx,xp,yp,y0,tableFunc(kind2,kind2),mag,col.red);
		}

		//caption
		if ( kind1!=kind2 ) {
			pfg.drawText(ctx," "+str1+" - "+str1, xp+160, yp+20,col.cyan);
			pfg.drawText(ctx," "+str1+" - "+str2, xp+160, yp+40,col.green);
			pfg.drawText(ctx," "+str2+" - "+str2, xp+160, yp+60,col.red);
		}
		pfg.drawText(ctx,str, xp+50, yBottom+45, col.gray);
	};

	pfg.drawTable = function(ctx,xp,yp,y0,tpp,mag,color) {
		const irMax=pfg.irMax;
		ctx.strokeStyle = color;
		ctx.beginPath();
		for (let ir=irMax; ir>0; ir--) {
			const a = tpp[ir]*mag;
			if (a>200.0) continue;
			ctx.lineTo(xp+ir*0.3, y0-a);
		}
		ctx.stroke();
	};

	//--- pfg utility

	pfg.drawText = function(ctx, txt, x, y, color) {
		ctx.fillStyle = color;
		ctx.fillText(txt, x, y);
	};

	pfg.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	// -------------------- pfg 3D module end


	// --------------------  3D graphics (particles) module  --------------------
	//
	// ver 0.0.1  2018.12.23  last updated on 2023.04.10
	// ver 0.0.2  2023.05.02  last updated on 2023.06.02

	let g_nParticles, g_xMax, g_yMax, g_zMax, g_xCanvasSize, g_yCanvasSize, g_xBoxSize, g_yShift; // gloval in g3d

	const g3d = {};				// namespace of graphic 3D module
	g3d.mouseDownFlag = 0;		// 1:on mouse down, 0:else
	g3d.x_mouse = 0;			// x-position of mouse
	g3d.y_mouse = 0;			// y-position of mouse
	g3d.x0_mouse = 0;			// drag-started x-position of mouse
	g3d.y0_mouse = 0;			// drag-started y-position of mouse
	g3d.zoom = 1.0;

	g3d.cx0 = 0.0;				// x-rotate center (3D graphics)
	g3d.cy0 = 0.0;				// y-rotate center (3D graphics)
	g3d.cz0 = 0.0;				// z-rotate center (3D graphics)
	g3d.Ax = -Math.PI/12.0;		// x-rotate angle around x-axis
	g3d.Ay = -Math.PI/12.0;		// y-rotate angle around y-axis
	g3d.ddAy = 0.0;				// Ay change rate for auto-rotate: eg. dday=0.5*Math.PI/180
	g3d.cosAx = 0.0;			// cosAx=Math.cos(Ax)
	g3d.sinAx = 0.0;			// sinAx=Math.sin(Ax)
	g3d.cosAy = 0.0;			// cosAy=Math.cos(Ay)
	g3d.sinAy = 0.0;			// sinAy=Math.sin(Ay)

	g3d.ppx = [];				// (m) rotated x-component of i-th particle position
	g3d.ppy = [];				// (m) rotated y-component of i-th particle position
	g3d.ppz = [];				// (m) rotated z-component of i-th particle position
	g3d.srtzix = [];			// z(depth)-sorted index: srtzix[1], srtzix[2],...,srtzix[g_nParticles]

	g3d.xApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// x-apex of box
	g3d.yApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// y-apex of box
	g3d.zApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// z-apex of box
	g3d.pxApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated x-apex of box
	g3d.pyApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated y-apex of box
	g3d.pzApex = [0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0];		// rotated z-apex of box
	g3d.boxApex = [[0,0,0], [1,0,0], [0,1,0], [1,1,0], [0,0,1], [1,0,1], [0,1,1], [1,1,1] ];
	g3d.boxEdge = [[0,1,9], [0,2,9], [0,4,9], [1,3,9], [1,5,9], [2,3,9],
				   [2,6,9], [3,7,9], [4,5,9], [4,6,9], [5,7,9], [6,7,9] ];

	//--- set mouse on canvas

	// g3d.setMouseOnCanvas( canvas );
	g3d.setMouseOnCanvas = function( canvas ) {
		canvas.addEventListener('mousemove', g3d.mouse_move);
		canvas.addEventListener('mousedown', g3d.mouse_down);
		canvas.addEventListener('mouseup', g3d.mouse_up);
		//canvas.addEventListener("mousewheel", g3d.mouseWheel);
	};
	g3d.setMouseOnCanvas.defined = false;

	g3d.mouse_move = function(e) {
		const pi = Math.PI;

		if (g3d.mouseDownFlag==1) {
			g3d.x_mouse = e.clientX;
			g3d.y_mouse = e.clientY;
			g3d.Ay = g3d.Ay + 0.5*(g3d.x_mouse-g3d.x0_mouse)*pi/180;
			if (g3d.Ay<-pi) g3d.Ay += 2*pi;
			if (g3d.Ay>pi) g3d.Ay += -2*pi;
			//g3d.Ax = g3d.Ax + 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			g3d.Ax = g3d.Ax - 0.5*(g3d.y_mouse-g3d.y0_mouse)*pi/180;
			if (g3d.Ax<-0.5*pi) g3d.Ax = -0.5*pi;
			if (g3d.Ax>0.5*pi) g3d.Ax = 0.5*pi;
			g3d.x0_mouse = g3d.x_mouse;
			g3d.y0_mouse = g3d.y_mouse;
		}
	};

	g3d.mouse_down = function(e) {
		if (g3d.mouseDownFlag==0) {
			g3d.x0_mouse = e.clientX;
			g3d.y0_mouse = e.clientY;
			g3d.x_mouse = g3d.x0_mouse;
			g3d.y_mouse = g3d.y0_mouse;
			g3d.mouseDownFlag = 1;
		}
	};

	g3d.mouse_up = function(e) {
		if (g3d.mouseDownFlag==1) {
			g3d.mouseDownFlag = 0;
		}
	};

	g3d.mouseWheel = function(e) {
		g3d.deltaY = e.deltaY;
		if ( g3d.deltaY > 0 ) g3d.zoom *= 0.95;
		else if ( g3d.deltaY < 0 ) g3d.zoom *= 1.05;
		if ( g3d.zoom<0.5 ) g3d.zoom = 0.5;
		if ( g3d.zoom>2.0 ) g3d.zoom = 2.0;
	};

	//--- 3D graphics aid (particles)

	// g3d.init( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift )
	g3d.init = function( nParticles, xMax, yMax, zMax, xCanvasSize, yCanvasSize, xBoxSize, yShift ) {
		g_nParticles = nParticles;
		g_xMax = xMax;
		g_yMax = yMax;
		g_zMax = zMax;
		g_xCanvasSize = xCanvasSize; g_yCanvasSize = yCanvasSize;
		g_xBoxSize = (xBoxSize==undefined) ? 300 : xBoxSize;
		g_yShift = (yShift==undefined) ? 20 : yShift;
		g3d.cx0 = 0.5*g_xMax;		// x-rotate center (3D graphics)
		g3d.cy0 = 0.5*g_yMax;		// y-rotate center (3D graphics)
		g3d.cz0 = 0.5*g_zMax;		// z-rotate center (3D graphics)
	}

	// g3d.setRotateAngle( AxInDegree, AyInDegree );
	g3d.setRotateAngle = function( AxInDegree, AyInDegree ) {
		g3d.Ax = AxInDegree*Math.PI/180.0;
		g3d.Ay = AyInDegree*Math.PI/180.0;
	};

	// g3d.scxpypFunc();
	g3d.scxpypFunc = function() {
		const xBoxSize = g_xBoxSize;
		const xCenter = xCanvasSize/2, yCenter = yCanvasSize/2-g_yShift, yBoxSize = (xBoxSize/g_xMax)*g_yMax;
		const xp = xCenter - (xBoxSize/2)*g3d.zoom, yp = yCenter - (yBoxSize/2)*g3d.zoom; // g3d param
		const sc = (xBoxSize/g_xMax)*g3d.zoom;
		return [ sc, xp, yp ];
	}

	g3d.set3DRotatedObjects = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate angle and rotate center(=box center)
		g3d.rotateXY();         // g_xx(i),g_yy(i),g_zz(i) rotate--> ppx(i),ppy(i),ppz(i)
		g3d.sortz();            // sort ppz(i) : ppz(srtzix(1)),ppz(srtzix(2)),...,ppz(srtzix(g_nParticles))
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.setBox = function() {
		for (let i=0; i<8; i++) {
			g3d.xApex[i] = g3d.boxApex[i][0]*g_xMax;
			g3d.yApex[i] = g3d.boxApex[i][1]*g_yMax;
			g3d.zApex[i] = g3d.boxApex[i][2]*g_zMax;
		}
	};

	g3d.setRotateXY = function(angleX,angleY) {
		g3d.cosAx = Math.cos(angleX);
		g3d.sinAx = Math.sin(angleX);
		g3d.cosAy = Math.cos(angleY);
		g3d.sinAy = Math.sin(angleY);
		g3d.cx0 = 0.5*g_xMax;
		g3d.cy0 = 0.5*g_yMax;
		g3d.cz0 = 0.5*g_zMax;
	};

	g3d.rotateXY = function() { //particles and box apex
		const nn = g_nParticles;
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<nn; i++) {
			let xi, yi, zi;
			[ xi, yi, zi ] = theModule.getPosition(i);
			const y = g_yMax-yi-cy0;
			g3d.ppx[i] = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
			g3d.ppy[i] = cosAx*y-sinAx*(zi-cz0) + cy0;
			g3d.ppz[i] =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		}
		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};

	g3d.rotateXYPos = function(xi,yi,zi) {
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;
		const y = g_yMax-yi-cy0;
		const rotx = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
		const roty = cosAx*y-sinAx*(zi-cz0) + cy0;
		//const rotz =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		return [ rotx, roty ];
	};

	g3d.sortz = function() {
		const nn=g_nParticles;
		for (let i=0; i<nn; i++) {
			g3d.srtzix[i] = i;
		}
		g3d.qSort(0,nn-1);
	};

	g3d.qSort = function(le,ri) {
		let i,j, pv,w;
		if (ri>le) {
			i = le-1;
			j = ri;
			pv = g3d.ppz[g3d.srtzix[ri]];
			while (1) {
				do {
					i=i+1;
				} while (pv>g3d.ppz[g3d.srtzix[i]]);
				do {
					j=j-1;
				} while (j>i && g3d.ppz[g3d.srtzix[j]]>pv);
				if (j<=i) break;
				w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[j]; g3d.srtzix[j]=w;
			}
			w=g3d.srtzix[i]; g3d.srtzix[i]=g3d.srtzix[ri]; g3d.srtzix[ri]=w;
			g3d.qSort(le,i-1);
			g3d.qSort(i+1,ri);
		}
	};

	g3d.markFarEdge = function() {
		//seek far apex --> iMin
		let zMin = g3d.pzApex[0];
		let iMin = 0;
		for (let i=1; i<8; i++) {
			if (zMin>g3d.pzApex[i]) {
				zMin = g3d.pzApex[i];
				iMin = i;
			}
		}
		//mark far edge
		for (let iEdge=0; iEdge<12; iEdge++) {
			g3d.boxEdge[iEdge][2] = 0;
			if (g3d.boxEdge[iEdge][0]==iMin || g3d.boxEdge[iEdge][1]==iMin) g3d.boxEdge[iEdge][2] = 1;
		}
	};

	g3d.plotNearEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==0) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotFarEdge = function(ctx, sc,xp,yp,color) {
		for (let iEdge=0; iEdge<12; iEdge++) {
			if (g3d.boxEdge[iEdge][2]==1) { //far edge mark == 1
				g3d.plotEdge(ctx, iEdge,sc,xp,yp,color);
			}
		}
	};

	g3d.plotEdge = function(ctx, iEdge,sc,xp,yp,color) {
		let iApex = g3d.boxEdge[iEdge][0];
		const x1 = g3d.pxApex[iApex]*sc+xp;
		const y1 = g3d.pyApex[iApex]*sc+yp;
		iApex = g3d.boxEdge[iEdge][1];
		const x2 = g3d.pxApex[iApex]*sc+xp;
		const y2 = g3d.pyApex[iApex]*sc+yp;
		g3d.drawLine(ctx, x1, y1, x2, y2, color);
	};

	g3d.drawLine = function(ctx, x1, y1, x2, y2, color) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.moveTo(x1, y1);
		ctx.lineTo(x2, y2);
		ctx.stroke();
	};

	g3d.drawDisc = function( ctx, x, y, r, color ) {
		ctx.fillStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.fill();
	};

	g3d.strokeCircle = function( ctx, x, y, r, color ) {
		ctx.strokeStyle = color;
		ctx.beginPath();
		ctx.arc(x, y, r, 0, 2*Math.PI, false);
		ctx.stroke();
	}

	// --------------------  3D graphics (particles) module end  --------------------

	// g3d_extension velocity space  created 2023.07.03, last updated 2023.07.03
	// g3d.setVspaceRotatedObjects(rotateRateOfAyInDegree)
	g3d.setVspaceRotatedObjects = function(rotateRateOfAyInDegree) {
		g3d.ddAy = rotateRateOfAyInDegree*Math.PI/180.0;
		g3d.Ay= g3d.Ay + g3d.ddAy; // auto-rotate : if (ddAy==0.0), stop
		if (g3d.Ay>Math.PI) g3d.Ay = g3d.Ay - 2.0*Math.PI;
		if (g3d.Ay<-Math.PI) g3d.Ay = g3d.Ay + 2.0*Math.PI;
		g3d.setBox();           // set box apex
		g3d.setRotateXY(g3d.Ax,g3d.Ay); // set rotate angle and rotate center(=box center)
		g3d.VspaceRotateXY();         // vx(i),vy(i),vz(i) rotate--> ppx(i),ppy(i),ppz(i)
		g3d.sortz();            // sort ppz(i) : ppz(srtzix(1)),ppz(srtzix(2)),...,ppz(srtzix(g_nParticles))
		g3d.markFarEdge();      // boxEdge[iEdge][2]=1:far side edge or 0:near side edge
	};

	g3d.VspaceRotateXY = function() {
		const nn = g_nParticles, vmag = 1.0;
		const cosAx=g3d.cosAx,sinAx=g3d.sinAx,cosAy=g3d.cosAy,sinAy=g3d.sinAy,cx0=g3d.cx0,cy0=g3d.cy0,cz0=g3d.cz0;

		for (let i=0; i<nn; i++) {
			let vxi, vyi, vzi; [ vxi, vyi, vzi ] = theModule.getVelocity(i);
			const xi = vxi*vmag+cx0, yi = vyi*vmag+cy0, zi = vzi*vmag+cz0;
			const y = g_yMax-yi-cy0;
			g3d.ppx[i] = cosAy*(xi-cx0)+sinAy*(sinAx*y+cosAx*(zi-cz0)) + cx0;
			g3d.ppy[i] = cosAx*y-sinAx*(zi-cz0) + cy0;
			g3d.ppz[i] =-sinAy*(xi-cx0)+cosAy*(sinAx*y+cosAx*(zi-cz0)) + cz0;
		}
		for (let i=0; i<8; i++) {
			g3d.pxApex[i] = cosAy*(g3d.xApex[i]-cx0)+sinAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cx0;
			g3d.pyApex[i] = cosAx*(g3d.yApex[i]-cy0)-sinAx*(g3d.zApex[i]-cz0) + cy0;
			g3d.pzApex[i] =-sinAy*(g3d.xApex[i]-cx0)+cosAy*(sinAx*(g3d.yApex[i]-cy0)+cosAx*(g3d.zApex[i]-cz0))+cz0;
		}
	};


	// --------------------  control  --------------------

	function initDom() {
		document.getElementById("step_button").style.visibility = "hidden";
	}

	function reset() { resetFlag = true; }

	function pause() {
		let btn = document.getElementById("pause_button");

		pauseFlag = ( pauseFlag==false );
		if ( pauseFlag==false ) btn.innerHTML = "pause"; else btn.innerHTML = "go";

		if ( pauseFlag==true ) {
			document.getElementById("step_button").style.visibility = "visible";
		} else {
			document.getElementById("step_button").style.visibility = "hidden";
		}
	}

	function step() { stepFlag = true; }

	function setTheme() {
		v_theme = 0 + document.getElementById("slct_theme").selectedIndex;
		resetFlag = true;
	}

	function setboundary() {
		v_boundary = 0 + document.getElementById("slct_boundary").selectedIndex;
		resetFlag = 1;
	}

	function setEnsemble() {  // select ensemble
		v_ensemble = 0 + document.getElementById("slct_ensemble").selectedIndex;
	}

	function setnParticles() {
		const n = 1 + document.getElementById("slct_nn").selectedIndex;
		v_nn = n*100;
		resetFlag = true;
	}

	function setTempMode() {
		v_tempMode = 0 + document.getElementById("slct_tempMode").selectedIndex;
	}

	function setContTemp() {
		v_contTemp = Number(document.getElementById("range_temp").value);
		document.getElementById("text_temp").innerHTML = " " + v_contTemp.toFixed(0);
	}

	function setContPress() {
		v_contPress = 0 + 1.0e6*document.getElementById("range_press").value;
		document.getElementById("text_press").innerHTML = " " + (v_contPress*1.0e-6).toFixed(1);
	}

	function setDispMode() {
		dispMode = 0 + document.getElementById("slct_dispMode").selectedIndex;
	}

	function setNcalc() {
		nCalc = 1 + document.getElementById("slct_nCalc").selectedIndex;
	}

	function viewHome() {
		g3d.setRotateAngle(-15,-15);
		g3d.zoom = 1.0;
	}

  // function controlled by python

  function breakLoop() {
    breakFlag = true;
  }

  function pysetTheme( theme ) {
    v_theme = theme
    document.getElementById("slct_theme").selectedIndex = theme;
    resetFlag = true;
  }

  function pysetTempMode( mode ) {
    v_tempMode = mode;
    document.getElementById("slct_tempMode").selectedIndex = mode;
  }

  function pysetTemperature(temp) {
    theModule.setTemp(temp);
  }

  function pysetDispMode( mode ) {
    dispMode = mode;
    document.getElementById("slct_dispMode").selectedIndex = mode;
  }

  function pygetData( pyMsg ) {
    document.getElementById("text_from_python").innerHTML = pyMsg;
    return [ sysTime, meanTemp, kineticEnergy, potentialEnergy, meanPress ];
  }

  function pygetParticlesList() {
    perticleFlag = false;
    return [ nowData, kindList, xxList, yyList, zzList, vxList, vyList, vzList ];
  }


	// --------------------  public  --------------------

	return {
		main:			main,			// main()

		// dom control
		reset:			reset,			// reset()
		pause:			pause,			// pause()
		step:			step,			// step()

		setTheme:		setTheme,		// setTheme()
		setboundary:	setboundary,	// setboundary()
		setEnsemble:	setEnsemble,	// setEnsemble()
		setnParticles:	setnParticles,	// setnParticles()
		setTempMode:	setTempMode,	// setTempMode()
		setContTemp:	setContTemp,	// setContTemp()
		setContPress:	setContPress,	// setContPress()
		setDispMode:	setDispMode,	// setDispMode()
		setNcalc:		setNcalc,		// setNcalc()
		viewHome:		viewHome,		// viewHome()

    breakLoop: breakLoop, // breakLoop();
    pysetTheme: pysetTheme, // pysetTheme( theme )
    pysetTempMode: pysetTempMode, // pysetTempMode( mode )
    pysetTemperature: pysetTemperature, // pysetTemperature( temp )
    pysetDispMode: pysetDispMode, // pysetDispMode( mode )
    pygetData: pygetData, // pygetData( pyMsg ) : return [ sysTime, meanTemp, kineticEnergy, potentialEnergy, meanPress ]
    pygetParticlesList: pygetParticlesList, //() :return [ nowData, kindList, xxList, yyList, zzList, vxList, vyList, vzList ]
	};

})(); // ====================  js058 module end  ====================


const js = js058;
//window.addEventListener('load', js.main );
js.main();


// %%%%%%%%%%%%%%%%%%%%  end of javaScript  %%%%%%%%%%%%%%%%%%%%

</script>

<style type="text/css">
    body { text-align:left; color:#000000; background-color:#fff8dd; }
</style>

</head>

<body>
<p>[js058] ensemble - Lennard-Jones molecular dynamics 2D</p>
<canvas ID="canvas_box" style="background-color: #000000;" width="480" height="480"></canvas>
<br>

<label>theme:</label>
<select id="slct_theme" onChange="js.setTheme()">
<option>Ar(fcc)</option><option>Kr(fcc)</option><option>Xe(fcc)</option>
<option>Ar(liq)</option><option>Kr(liq)</option><option>Xe(liq)</option>
<option>Ne</option><option selected>Ar</option><option>Kr</option><option>Xe</option><option>Hg</option>
<option>Ar-Xe</option><option>Ar-Hg</option><option>Ne-Kr</option><option>Ne-Ar</option>
</select>
    <span style="margin-right:10px;"></span>
<label>boundary:</label>
<select id="slct_boundary" onChange="js.setboundary()">
<option selected>wall</option><option>periodic</option>
</select>
    <span style="margin-right:10px;"></span>
<label>ensemble:</label>
<select id="slct_ensemble" onChange="js.setEnsemble()">
<option>NVE adiabatic</option><option>NVT contT</option><option>NPT contT/P</option>
</select>
<br>

<label>nCalc/frame:</label>
<select id="slct_nCalc" onChange="js.setNcalc()">
<option selected>1</option><option>2</option><option>3</option><option>4</option>
</select>
    <span style="margin-right: 190px;"></span>
<button onClick="js.reset()">reset</button>
    <span style="margin-right: 20px;"></span>
<button id="pause_button" onClick="js.pause()">pause</button>
    <span style="margin-right: 10px;"></span>
<button id="step_button" onClick="js.step()">step</button>
<br>

<label>cont. Temp :</label>
<input type="range" id="range_temp" min="10" max="600" value="300" step="2"
style="width:360px" oninput="js.setContTemp()">
<span id="text_temp"> 300</span>
<br>

<label>cont. Press:</label>
<input type="range" id="range_press" min="0" max="100" value="10" step="1"
style="width:360px" oninput="js.setContPress()">
<span id="text_press"> 10.0</span>
<br>

<label>disp mode:</label>
<select id="slct_dispMode" onChange="js.setDispMode()">
<option selected>ball</option><option>small ball</option>
<option>small ball + bond</option><option>bond only</option>
<option>small ball + velocity</option><option>velocity space ( full scale=1500 m/s )</option>
<option>velocity distribution function</option><option>radial distribution function</option>
<option>force F(r)</option><option>potential V(r)</option>
<!-- <option>fig.9</option><option>fig.10</option> -->
</select>
    <span style="margin-right: 70px;"></span>
<button onClick="js.viewHome()">view home</button>
<br>

<p id="text_caption" ></p>
<hr width="480" align="left" color="#a0a0a0">
<button onClick="js.breakLoop()">animation break to END</button>
    <span style="margin-right: 50px;"></span> python msg:
<span id="text_from_python" ></span>
<br>

</body>
</html>


  ''')
  display(htm)
# end def


In [ ]:
# exec html-js code
exec_html_js()
print("--- push [animation break to END] button to end ---")

In [ ]:
# get data and print

import time

# exec html-js code
exec_html_js()
print("-- start --")

# get data and print
for i in range(10):
  [ sysTime, sysTemp, kineticEnergy, potentialEnergy, sysPress ] = eval_js( 'js.pygetData({})'.format(i) )
  energy = kineticEnergy + potentialEnergy
  print( f'i = {i:>2d},  time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  press = {sysPress:>7.3e} (Pa),  energy = {energy:9.6e} (J)' )
  time.sleep(2)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# change theme and dispMode

import time

themeList = [
    "0:Ar(fcc)", "1:Kr(fcc)", "2:Xe(fcc)", "3:Ar(liq)", "4:Kr(liq)", "5:Xe(liq)",
    "6:Ne", "7:Ar", "8:Kr", "9:Xe", "10:Hg", "11:Ar-Xe", "12:Ar-Hg", "13:Ne-Kr", "14:Ne-Ar" ]
dispModeList = [
    '0: ball', '1: small ball', '2: small ball + bond', '3: bond only', '4: small ball + velocity',
    '5: velocity space ( full scale=1500 m/s )', '6: velocity distribution function', '7: radial distribution function',
    '8: force F(r)', '9: potential V(r)' ]

# exec html-js code
exec_html_js()
print("-- start --")

# change theme and dispMode
for theme in [ 0, 3, 7, 14 ]:
  eval_js( 'js.pysetTheme({})'.format(theme) )
  print( "-- theme:", themeList[theme], "--" )
  for dispMode in [ 0, 2, 5, 6, 9 ]:
    eval_js( 'js.pysetDispMode({})'.format(dispMode) )
    print( "-- -- dispMode:", dispModeList[dispMode], "--" )
    time.sleep(3)
    # get data and display
    [ sysTime, sysTemp, kineticEnergy, potentialEnergy, sysPress ] = eval_js( 'js.pygetData({})'.format(theme) )
    energy = kineticEnergy + potentialEnergy
    print( f'-- -- time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  press = {sysPress:>7.3e} (Pa),  energy = {energy:9.6e} (J)' )
    time.sleep(2)


# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# system cool down

import time
import matplotlib.pyplot as plt

# exec html-js code
exec_html_js()
print("-- start --")

# set theme
theme = 7 # Ar gas
eval_js( 'js.pysetTheme({})'.format(theme) )
print("-- theme: Ar --")

# change dispMode
eval_js( 'js.pysetDispMode(2)' ) # 2:'small ball + bond'
print("-- change dispMode=2:'small ball + bond' --")

timeList = []
tempList = []
energyList = []
kkList = []
uuList = []

for i in range(20):
  # set temperature
  temp = max(200 - 10*i, 10)
  eval_js( 'js.pysetTemperature({})'.format(temp) )
  time.sleep(3)

  # get data and display
  [ sysTime, sysTemp, kk, uu, sysPress ] = eval_js( 'js.pygetData({})'.format(theme) )
  energy = kk + uu

  timeList.append(sysTime*1e12)
  tempList.append(sysTemp)
  energyList.append(energy*1e20)
  kkList.append(kk*1e20)
  uuList.append(uu*1e20)

  if i%2==0:
    print( f'-- -- time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  press = {sysPress:>7.3e} (Pa),  energy = {energy:9.6e} (J)' )
  time.sleep(3)

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- simulator stop --")

# plot temp, energy, K, U
plt.plot( timeList, tempList, 'or-' )
plt.xlabel('time (ps)')
plt.ylabel('temp (K)')
plt.show()

plt.plot( timeList, energyList, 'or-' )
plt.plot( timeList, kkList, '^g-' )
plt.plot( timeList, uuList, 'xb-' )
plt.xlabel('time (ps)')
plt.ylabel('energy, K, U (x 1e-20 J)')
plt.show()

In [ ]:
# get particles data

import time

# exec html-js code
exec_html_js()
print("-- start --")

# change theme
theme = 14 # "14:Ne-Ar"
eval_js( 'js.pysetTheme({})'.format(theme) )
print( "-- theme:", themeList[theme], "--" )

time.sleep(10)

# get particle data
print("-- get particles data --")
[ nowData, kindList, xxList, yyList, zzList, vxList, vyList, vzList ] = eval_js('js.pygetParticlesList()')
[ sysTime, sysTemp, kineticEnergy, potentialEnergy, sysPress ] = nowData
energy = kineticEnergy + potentialEnergy
print( f'-- time = {sysTime*1e12:>7.2f} (ps),  temp = {sysTemp:>6.1f} (K),  press = {sysPress:>7.3e} (Pa),  energy = {energy:9.6e} (J)' )
print(f'len(kindList) ={len(kindList):>4d}, len(xxList) ={len(xxList):>4d}, len(yyList) ={len(yyList):>4d}, len(zzList) ={len(zzList):>4d}')
print(f'len(vxList) ={len(vxList):>4d}, len(vyList) ={len(vyList):>4d}, len(vzList) ={len(vzList):>4d}')

# animation break to END
eval_js( 'js.breakLoop()' )
print("-- end --")

In [ ]:
# save particles data

import numpy as np

# convert [ kindList, xxList, yyList, zzList, vxList, vyList, vzList ] to np_data
np_data = np.array([ kindList, xxList, yyList, zzList, vxList, vyList, vzList ])
print("shape of  np_data :", np_data.shape )

# save np_data
print("-- save as 'js058_data.npy'" )
np.save( 'js058_data.npy', np_data )

In [ ]:
# load data and prepare numpy data for plot

import numpy as np
import matplotlib.pyplot as plt

# atom data            'He'              'Ne'              'Ar'              'Kr'              'Xe               'Hg'
colorList = np.array([ [0xff,0x50,0x50], [0xff,0xb4,0x00], [0xb4,0xff,0x00], [0x50,0xff,0x50], [0x50,0xb4,0xf], [0x80,0x00,0xff] ]) / 255
sizeList = np.array([ 2.57, 2.97, 3.42, 3.62, 4.06, 2.90 ])  # He, He, Ar, Kr, Xe, Hg
atomList = [ 'He', 'Ne', 'Ar',  'Kr', 'Xe', 'Hg' ]

# load np_data <-- np.array([ kindList, xxList, yyList, zzList, vxList, vyList, vzList ])
print("-- load data --")
loaded_data = np.load('js058_data.npy')

# prepare data ( numpy array ) for plot
Kind = loaded_data[0].astype(int)  # kind of every atom
X = loaded_data[1] * 1e9           # (nm) x-position of every atom
Y = loaded_data[2] * 1e9           # (nm) y-position of every atom
Z = loaded_data[3] * 1e9           # (nm) z-position of every atom
U = loaded_data[4]                 # (m/s) x-component of velocity of every atom
V = loaded_data[5]                 # (m/s) y-component of velocity of every atom
W = loaded_data[6]                 # (m/s) z-component of velocity of every atom
C = colorList[Kind]                # color of ball representation of every atom
S = sizeList[Kind]                 # size of ball

In [ ]:
# Ne-Ar in the box

import numpy as np
import matplotlib.pyplot as plt

fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')
ax.scatter(X, Y, Z, c=C, s=S*10)
plt.title('Ne-Ar in the box')
plt.show()

In [ ]:
# Ne-Ar in the box - plotly

import numpy as np
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=X, y=Y, z=Z,
    mode='markers',
    marker=dict( color=C, size=S*3, ),
))

fig.update_layout(
    title='Ne-Ar in the box',
    width=800, height=800,
    scene=dict(
        xaxis_title='vx', yaxis_title='vy', zaxis_title='vz', ),
)

fig.show()

In [ ]:
# plot atoms in velocity space

import numpy as np
import matplotlib.pyplot as plt

# scatter plot
fig = plt.figure(figsize=(6, 6))
ax = fig.add_subplot(projection='3d')
ax.scatter(U, V, W, c=C, s=S)
plt.title('Ne-Ar in velocity space')
plt.show()

In [ ]:
# plot atoms in velocity space - plotly

import numpy as np
import plotly.graph_objects as go

fig = go.Figure()
fig.add_trace(go.Scatter3d(
    x=U, y=V, z=W,
    mode='markers',
    marker=dict( color=C, size=S, ),
))

fig.update_layout(
    title='Ne-Ar in velocity space',
    width=800, height=800,
    scene=dict(
        xaxis_title='vx', yaxis_title='vy', zaxis_title='vz', ),
)

fig.show()

In [ ]:
# histogram of |velocity|

import numpy as np
import matplotlib.pyplot as plt

Vabs = np.sqrt(U**2 + V**2 + W**2)

plt.hist(Vabs, 30)
plt.title('the histogram of |velocity|')
plt.xlabel('|v|  (m/s)')
plt.show()

In [ ]:
# histogram of vx (Ne and Ar)

import numpy as np
import matplotlib.pyplot as plt

U_Ne = [vx for vx, k in zip(U,Kind) if k==1]  # kind of Ne: k==1
U_Ar = [vx for vx, k in zip(U,Kind) if k==2]  # kind of Ar: k==2

fig = plt.figure(figsize=(12, 6))

ax1 = fig.add_subplot(1, 2, 1)
ax1.hist(U_Ne, 30)
ax1.set_title(f'the histogram of vx (Ne)  ( N = {len(U_Ne)} )' )
ax1.set_xlabel('vx  (m/s)')

ax2 = fig.add_subplot(1, 2, 2)
ax2.hist(U_Ar, 30)
ax2.set_title(f'the histogram of vx (Ar)  ( N = {len(U_Ar)} )' )
ax2.set_xlabel('vx  (m/s)')

plt.show()